# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [30]:
import boto3
import json

In [31]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [36]:
%store -r

Please note these steps rely on each step being fully completed before the other will work. If you receive an error, wait a minute or two and try again.

## Clean up Campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will delete the Campaigns you created.

In [33]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

{'ResponseMetadata': {'RequestId': '57b7770d-eb36-4a2e-88f8-95cbec691f8e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Sep 2020 14:57:44 GMT',
   'x-amzn-requestid': '57b7770d-eb36-4a2e-88f8-95cbec691f8e',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up event trackers

Next, clean up any event trackers. The code below will delete your event tracker.

In [37]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

{'ResponseMetadata': {'RequestId': 'fe2621df-b587-4455-9f02-48a4942749ca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Sep 2020 14:58:17 GMT',
   'x-amzn-requestid': 'fe2621df-b587-4455-9f02-48a4942749ca',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up Filters

Next, clean up any filters. The code below iterate over all filters you created and delete them. Note, if you get an error about a campaign, that is fine, wait for 2 minutes then try the cell below again. 

In [39]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## Clean up solutions

Next, clean up the solutions. The code below will delete the solutions that were created.

In [40]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

{'ResponseMetadata': {'RequestId': 'd3a1d2b9-a5d2-4389-a073-4b8125d2d7e5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Sep 2020 15:16:24 GMT',
   'x-amzn-requestid': 'd3a1d2b9-a5d2-4389-a073-4b8125d2d7e5',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up datasets

Next, clean up the datasets. The code below will delete the item and interactions datasets.

In [41]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

{'ResponseMetadata': {'RequestId': 'cfdf8da1-e41f-4945-8a6e-eb9160695b3e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Sep 2020 15:16:26 GMT',
   'x-amzn-requestid': 'cfdf8da1-e41f-4945-8a6e-eb9160695b3e',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the schemas

Next, clean up the schemas. You create an interaction and item schema, delete them.

In [43]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

{'ResponseMetadata': {'RequestId': '48d48a43-89bb-49f6-bd6a-1a33b814e261',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 10 Sep 2020 15:16:49 GMT',
   'x-amzn-requestid': '48d48a43-89bb-49f6-bd6a-1a33b814e261',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the dataset groups

Finally, clean up the dataset group:

In [44]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

ResourceInUseException: An error occurred (ResourceInUseException) when calling the DeleteDatasetGroup operation: The provided dataset-group is being referenced by one or more solutions

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [25]:
iam = boto3.client('iam')

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [26]:
iam.list_attached_role_policies(
    RoleName = role_name
)

{'AttachedPolicies': [{'PolicyName': 'AmazonS3FullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/AmazonS3FullAccess'},
  {'PolicyName': 'AmazonPersonalizeFullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '670a0678-e232-42bb-958c-f631ee7bcfbd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '670a0678-e232-42bb-958c-f631ee7bcfbd',
   'content-type': 'text/xml',
   'content-length': '728',
   'date': 'Wed, 09 Sep 2020 22:59:46 GMT'},
  'RetryAttempts': 0}}

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [27]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

{'ResponseMetadata': {'RequestId': '4b4d9a1d-cc5a-4e72-8c0d-7a3af92748ce',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4b4d9a1d-cc5a-4e72-8c0d-7a3af92748ce',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 09 Sep 2020 22:59:59 GMT'},
  'RetryAttempts': 0}}

In [28]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

{'ResponseMetadata': {'RequestId': '7c5e57ca-56f8-4e5b-bb2b-bd9ae0f1873b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7c5e57ca-56f8-4e5b-bb2b-bd9ae0f1873b',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Wed, 09 Sep 2020 23:00:03 GMT'},
  'RetryAttempts': 0}}

Finally, you should be able to delete the IAM role.

In [29]:
iam.delete_role(
    RoleName = role_name
)

{'ResponseMetadata': {'RequestId': 'e86892b1-859e-4b08-8025-0813966808b0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e86892b1-859e-4b08-8025-0813966808b0',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Wed, 09 Sep 2020 23:00:06 GMT'},
  'RetryAttempts': 0}}

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.